In [1]:
import os
import sys
import numpy as np
import csv
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
df_train = pd.read_pickle("./train_mr.pkl")
df_test = pd.read_pickle("./test_mr.pkl")
pd.set_option('display.max_columns', 500)

In [3]:
df_train.head()

,acquirer,bank,card,money,trade_cat,coin,online,trade_type,fallback,3ds,fraud_ind,pay_type,install,term,date,time,mcc,shop,excess,city,nation,status,key
0,6716,12765,101514,966.580017,4,62,0,2,0.0,0.0,0,5,1,2,1,29.0,247,34949,0,5817,102,0,1119159
1,5575,34837,60869,1071.099976,5,62,1,8,0.0,0.0,0,5,0,0,1,40.0,203,33794,0,2138,44,0,641444
2,6769,22630,110573,438.209991,5,62,0,5,0.0,0.0,0,5,0,0,1,45.0,251,79257,0,3588,102,0,188670
3,6767,91588,208961,1174.170044,5,62,0,5,0.0,0.0,0,5,0,0,1,48.0,293,14765,0,5817,102,0,1475737
4,3198,84148,180103,367.290009,5,60,0,5,0.0,0.0,0,5,0,0,1,51.0,251,68966,0,3982,38,0,641443


# Feature Engineering

## 1

In [4]:
#money / term groupby high_category: max, min, mean, std, var, median, median_diff, quantiles
def money_term_inf(dataframe,c,b):
    #b grouped by c
    a = dataframe.copy()
    a[b + '_' + c + '_max'] = a[c].map(a.loc[:, [b, c]].groupby(c).max().loc[:, b].to_dict())
    a[b + '_' + c + '_min'] = a[c].map(a.loc[:, [b, c]].groupby(c).min().loc[:, b].to_dict())
    a[b + '_' + c + '_mean'] = a[c].map(a.loc[:, [b, c]].groupby(c).mean().loc[:, b].to_dict())
    a[b + '_' + c + '_std'] = a[c].map(a.loc[:, [b, c]].groupby(c).std().loc[:, b].to_dict())
    a[b + '_' + c + '_var'] = a[b + '_' + c + '_std']**2
    a[b + '_' + c + '_median'] = a[c].map(a.loc[:, [b, c]].groupby(c).median().loc[:, b].to_dict())
    a[b + '_' + c + '_median_diff'] = a[b] - a[b + '_' + c + '_median']
    return a

## 2

In [5]:
#card_num: one bank has how many cards. ( 一個帳號有幾張卡 )
def card_num(dataframe):
    a = dataframe.copy()
    a['card_num'] = a['bank'].map(a.loc[:, ['card', 'bank']].groupby('bank').count().loc[:,'card'].to_dict())
    return a

## 4

In [11]:
#dominant coin type 貨幣在那個國家交易過幾次
def coin_trade_country_per(dataframe):
    a = dataframe.copy()
    dct =(a.groupby(['nation','coin']).size()/a.groupby('nation').size()).to_dict()
    a['coin_country_per'] = a.set_index(['nation', 'coin']).index.map(dct.get)
    return a

## 9

In [7]:
#money / term => term_money 分期金額
def money_divide_term(dataframe):
    a = dataframe.copy()
    a['money_term'] = a['money']/(a['term']+1)
    return a

## 11

In [26]:
#一個國家有幾個城市
def count_city_num(dataframe):
    a = dataframe.copy()
    a['city_num'] = a['city_num'] = a['nation'].map(a.loc[:, ['city', 'nation']].groupby('nation').count().loc[:,'city'].to_dict())
    return a

# 跑Function

In [8]:
#跑money_term_inf
high_category = ['acquirer', 'bank', 'card', 'coin', 'mcc', 'shop', 'city', 'nation']
for i in (high_category):
    df_train  =money_term_inf(df_train,i,'money')
    df_train = money_term_inf(df_train,i,'term')

In [9]:
#跑card_num
df_train = card_num(df_train)

In [12]:
#跑coin_trade_country_per
df_train = coin_trade_country_per(df_train)

In [13]:
#跑money_divide_term
df_train = money_divide_term(df_train)

In [27]:
#跑count_city_num
df_train = count_city_num(df_train)

In [30]:
df_train.head()

,acquirer,bank,card,money,trade_cat,coin,online,trade_type,fallback,3ds,fraud_ind,pay_type,install,term,date,time,mcc,shop,excess,city,nation,status,key,money_acquirer_max,money_acquirer_min,money_acquirer_mean,money_acquirer_std,money_acquirer_var,money_acquirer_median,money_acquirer_median_diff,term_acquirer_max,term_acquirer_min,term_acquirer_mean,term_acquirer_std,term_acquirer_var,term_acquirer_median,term_acquirer_median_diff,money_bank_max,money_bank_min,money_bank_mean,money_bank_std,money_bank_var,money_bank_median,money_bank_median_diff,term_bank_max,term_bank_min,term_bank_mean,term_bank_std,term_bank_var,term_bank_median,term_bank_median_diff,money_card_max,money_card_min,money_card_mean,money_card_std,money_card_var,money_card_median,money_card_median_diff,term_card_max,term_card_min,term_card_mean,term_card_std,term_card_var,term_card_median,term_card_median_diff,money_coin_max,money_coin_min,money_coin_mean,money_coin_std,money_coin_var,money_coin_median,money_coin_median_diff,term_coin_max,term_coin_min,term_coin_mean,term_coin_std,term_coin_var,term_coin_median,term_coin_median_diff,money_mcc_max,money_mcc_min,money_mcc_mean,money_mcc_std,money_mcc_var,money_mcc_median,money_mcc_median_diff,term_mcc_max,term_mcc_min,term_mcc_mean,term_mcc_std,term_mcc_var,term_mcc_median,term_mcc_median_diff,money_shop_max,money_shop_min,money_shop_mean,money_shop_std,money_shop_var,money_shop_median,money_shop_median_diff,term_shop_max,term_shop_min,term_shop_mean,term_shop_std,term_shop_var,term_shop_median,term_shop_median_diff,money_city_max,money_city_min,money_city_mean,money_city_std,money_city_var,money_city_median,money_city_median_diff,term_city_max,term_city_min,term_city_mean,term_city_std,term_city_var,term_city_median,term_city_median_diff,money_nation_max,money_nation_min,money_nation_mean,money_nation_std,money_nation_var,money_nation_median,money_nation_median_diff,term_nation_max,term_nation_min,term_nation_mean,term_nation_std,term_nation_var,term_nation_median,term_nation_median_diff,card_num,coin_country_per,money_term,city_num
0,6716,12765,101514,966.580017,4,62,0,2,0.0,0.0,0,5,1,2,1,29.0,247,34949,0,5817,102,0,1119159,4307.660156,0.0,805.271362,449.041382,201638.162601,722.380005,244.200012,8,0,0.248020,0.794906,0.631876,0,2,1248.109985,123.239998,800.699097,384.506927,147845.577288,966.580017,0.000000,2,0,1.090909,0.94388,0.890909,1.0,1.0,1248.109985,123.239998,784.799011,401.475952,161182.940153,966.580017,0.000000,2,0,1.1,0.994429,0.988889,1.5,0.5,7208.770020,0.0,682.752686,410.813324,168767.587155,644.659973,321.920044,8,0,0.060374,0.401904,0.161527,0,2,4910.180176,0.000000,765.138977,376.367615,141652.581430,708.229980,258.350037,8,0,0.188447,0.682097,0.465256,0,2,3140.280029,438.209991,923.810547,265.421509,70448.577328,865.039978,101.540039,7,0,1.476157,0.911971,0.831691,1.0,1.0,7208.770020,0.000000,680.749023,390.326477,152354.758687,636.820007,329.760010,8,0,0.029591,0.261273,0.068264,0.0,2.0,7208.770020,0.0,673.943787,387.901794,150467.802125,607.219971,359.360046,8,0,0.057499,0.392571,0.154112,0,2,11,0.858183,322.193329,1313475
1,5575,34837,60869,1071.099976,5,62,1,8,0.0,0.0,0,5,0,0,1,40.0,203,33794,0,2138,44,0,641444,3089.750000,0.0,641.221802,593.991699,352826.138741,618.184998,452.914978,0,0,0.000000,0.000000,0.000000,0,0,1535.599976,168.199997,615.295532,304.744781,92869.381848,513.799988,557.299988,0,0,0.000000,0.00000,0.000000,0.0,0.0,1535.599976,1071.099976,1303.349976,328.451111,107880.132212,1303.349976,-232.250000,0,0,0.0,0.000000,0.000000,0.0,0.0,7208.770020,0.0,682.752686,410.813324,168767.587155,644.659973,426.440002,8,0,0.060374,0.401904,0.161527,0,0,4035.129883,0.000000,1178.297852,743.321838,552527.355411,1216.844971,-145.744995,4,0,0.134211,0.534582,0.285778,0,0,2951.280029,0.000000,613.450684,617.167969,380896.301651,565.219971,505.880005,0,0,0.000000,0.000000,0.000000,0.0,0.0,2951.280029,0.000000,613.549072,616.545715,380128.619094,566.229980,504.869995,0,0,0.000000,0.000000

In [25]:
(tmp.loc[:, ['city', 'nation']].groupby('nation').count().loc[:,'city'].to_dict())

{0: 131,
 1: 536,
 2: 275,
 3: 1,
 4: 39,
 5: 164,
 6: 1514,
 7: 5,
 8: 3,
 9: 1,
 10: 242,
 11: 4,
 12: 1,
 13: 3,
 14: 119,
 15: 3,
 16: 1024,
 17: 665,
 18: 1,
 19: 7,
 20: 17882,
 22: 3,
 24: 1,
 25: 619,
 26: 235,
 27: 3041,
 28: 43,
 29: 1,
 30: 88,
 31: 13,
 32: 1029,
 33: 2,
 34: 118,
 35: 63,
 36: 1603,
 37: 2,
 38: 44311,
 39: 12,
 40: 106,
 41: 31,
 42: 4346,
 43: 2,
 44: 4191,
 45: 146,
 46: 41677,
 47: 38,
 48: 79,
 49: 405,
 50: 443,
 51: 3,
 52: 15537,
 53: 17,
 54: 45,
 55: 2195,
 56: 181,
 57: 1,
 59: 1,
 60: 15,
 61: 54,
 62: 538,
 63: 8,
 64: 6,
 65: 2,
 67: 20,
 68: 365,
 69: 32,
 70: 11,
 71: 2,
 72: 77,
 73: 603,
 74: 9,
 75: 20128,
 76: 118,
 77: 5,
 78: 291,
 79: 1,
 80: 8,
 81: 67,
 82: 8,
 83: 209,
 84: 2,
 85: 154,
 86: 1,
 87: 129,
 88: 1,
 89: 128,
 90: 20,
 91: 11,
 92: 1832,
 93: 5222,
 94: 102,
 95: 27,
 96: 13,
 97: 1,
 98: 840,
 99: 1,
 100: 1,
 101: 83,
 102: 1313475,
 104: 33496,
 105: 7,
 106: 366,
 107: 50}